In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import os

c:\Users\micha\anaconda3\envs\UVACapstone\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
_patents_full = pd.read_csv('./data/merged/PATENTS_DATA.csv', dtype = {'GEOID':'str'}, low_memory = False)

In [3]:
_ei = pd.read_csv('./data/ei/ei_final_final_hopeful.csv', dtype = {'GEOID':'str', 'County Code':'str'}, low_memory = False)

In [4]:
# check for duplicates in _ei
_ei[_ei[['County Code' , 'year']].duplicated()][['County Code' , 'year']].sort_values(['County Code' , 'year']).groupby('County Code').count()

,year
County Code,
11001,11
32510,8
45083,11


In [8]:

#  45085 is sumter SC
# for all rows in ei, if GEOID is 45085, then update with rows that were assigned to 45083
ei = _ei.copy()
for i  in ei.query('`County Code` == "45085"').index:
    year = ei.loc[i, 'year']
    results = ei.query(f'`County Code` == "45083" & year == {year} & County_Name == "Sumter, SC"')[['GEOID','County_Name','Year','Award Amount']].values.tolist()
    GEOID_tmp  = results[0][0]
    County_Name_tmp  = results[0][1]
    Year_tmp  = results[0][2]
    Award_Amount_tmp  = results[0][3]

    ei.loc[i, 'GEOID'] = GEOID_tmp
    ei.loc[i, 'County_Name'] = County_Name_tmp
    ei.loc[i, 'Year'] = Year_tmp
    ei.loc[i, 'Award Amount'] = Award_Amount_tmp
    

# Drop 45083 that are now 45085. That is to say, drop all 45083 that have "County_Name" == "Sumter, SC"
ei = ei.drop(ei[(ei['County Code'] == "45083") & (ei['County_Name'] == "Sumter, SC")].index)
ei[(ei['County Code'] == "45083") & (ei['County_Name'] == "Sumter, SC")].index


Int64Index([], dtype='int64')

In [9]:
# set max display rows and columns to 1000
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)


In [10]:
# I'm just dropping duplicates here. I'm not sure if this is the best way to do it.
# ei = ei.drop(ei.duplicated(subset = ['County Code', 'year']), axis = 0)
ei.drop_duplicates(subset = ['County Code', 'year'], inplace = True)


In [11]:
# check for duplicates in ei
# See how we did.
ei[ei[['County Code' , 'year']].duplicated()][['County Code' , 'year']].sort_values(['County Code' , 'year']).groupby('County Code').count()

,year
County Code,


In [12]:
# How many missing values are there in the county_GEOID_cross_df?
# That is to say, how many rows in bea are missing rows in the county crosswalk?
# test_merge = pd.merge(_patents_full, ei,
#                       left_on = ['GEOID', 'application_year'],
#                       right_on = ['County Code', 'year'],
#                       suffixes=('', '_ei'),
#                       how = 'outer',
#                       indicator = 'matched',
#                       validate = 'many_to_one')



# print(len(test_merge.query("matched == 'left_only'").GEOID.unique()))
# print(test_merge.query("matched == 'left_only'").GEOID.unique())

In [15]:
final_merge = pd.merge(_patents_full, ei,
                      left_on = ['GEOID', 'application_year'],
                      right_on = ['County Code', 'year'],
                      suffixes=('', '_ei'),
                      how = 'left')

In [21]:
# Clean up some columns
try:
    final_merge.drop(["Unnamed: 0","GeoFIPS","GeoName","year_ei","STNAME","CTYNAME","State","County","County Code","fipstate","fipscty","GEOID_ei","County_Name","Year"], axis = 1, inplace = True)
except:
    pass

# make the column names like the other column names
final_merge = final_merge.rename(columns={
    "Earnings by place of work" : "Earnings_by_place_of_work",
    "Employee and self-employed contributions for government social insurance" : "Employee_and_self-employed_contributions_for_government_social_insurance",
    "Employer contributions for employee pension and insurance funds 8" : "Employer_contributions_for_employee_pension_and_insurance_funds",
    "Employer contributions for government social insurance" : "Employer_contributions_for_government_social_insurance",
    "Equals: Net earnings by place of residence" : "Equals_Net_earnings_by_place_of_residence",
    "Farm income 2" : "Farm_income",
    "Farm proprietors' income" : "Farm_proprietors_income",
    "Less: Contributions for government social insurance 5" : "Less_Contributions_for_government_social_insurance",
    "Nonfarm personal income 1" : "Nonfarm_personal_income",
    "Nonfarm proprietors' income" : "Nonfarm_proprietors_income",
    "Per capita personal income (dollars) 4" : "Per_capita_personal_income_dollars",
    "Personal income (thousands of dollars)" : "Personal_income_thousands_of_dollars",
    "Plus: Adjustment for residence 6" : "Plus_Adjustment_for_residence",
    "Plus: Dividends, interest, and rent 7" : "Plus_Dividends_interest_and_rent",
    "Plus: Personal current transfer receipts" : "Plus_Personal_current_transfer_receipts",
    "Population (persons) 3" : "Population_persons",
    "Proprietors employment" : "Proprietors_employment",
    "Proprietors' income 9" : "Proprietors_income",
    "Supplements to wages and salaries" : "Supplements_to_wages_and_salaries",
    "Total employment" : "Total_employment",
    "Wage and salary employment" : "Wage_and_salary_employment",
    "Wages and salaries" : "Wages_and_salaries",
    "Population Estimate" : "Population_Estimate",
    "Applications" : "Applications",
    "Award Amount" : "Award_Amount"
})

In [23]:
# Save the final merge
final_merge.to_csv('./data/merged/PATENTS_DATA.csv', index = False)